In [1]:
import pygrib as pg
import numpy as np
# import xarray as xr
# import cfgrib
import os
import shutil

# 测试

In [3]:
def fetch_testfile(year, month, day, hour, target_path):
    
    if month == 7:
        monthname = 'Jul'
    elif month == 9:
        monthname = 'Sep'
    elif month == 10:
        monthname = 'Oct'
        
    filename = f"fnl_{year}{month:02d}{day:02d}_{hour:02d}_00.grib2"
    source_path= f"F:/Data/Graduation/fnl_months/{monthname}/{year}{month:02d}/{filename}"
    
    if not os.path.exists(target_path):
        os.makedirs(target_path)
    
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)
        print(f"{filename} copied to target_path")
        
        return filename
    else:
        raise FileNotFoundError(f"{source_path} not found")


file_list = []
years = range(2014,2023)
month = 9

day = 1
hour = 0

target_path='F:/Data/Graduation/fnl_months/test/'
print(target_path)

for year in years:
    try:
        file_name = fetch_testfile(year,month,day,hour,target_path)
        file_list.append(target_path+file_name)
    except FileNotFoundError as e:
        print(e)

In [4]:
def get_fnl(year,month,day,hour):
    
    if month == 7:
        monthname = 'Jul'
    elif month == 9:
        monthname = 'Sep'
    elif month == 10:
        monthname = 'Oct'
        
    filename = f"fnl_{year}{month:02d}{day:02d}_{hour:02d}_00.grib2"
    # path = f"F:/Data/Graduation/fnl_months/{monthname}/{year}{month:02d}/{filename}"
    path = f"F:/Data/Graduation/fnl_months/test/{filename}"
    
    if os.path.exists(path):
        return path
    else:
        raise FileNotFoundError(f"{path} not found")
    
file_list = []
years = range(2014,2023)
month = 9
day = 1
hour = 0

for year in years:
    try:
        file_path = get_fnl(year,month,day,hour)
        file_list.append(file_path)
    except FileNotFoundError as e:
        print(e)
file_list

['F:/Data/Graduation/fnl_months/test/fnl_20140901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20150901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20160901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20170901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20180901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20190901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20200901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20210901_00_00.grib2',
 'F:/Data/Graduation/fnl_months/test/fnl_20220901_00_00.grib2']

## 读取单个文件的变量

In [52]:
grb_in = pg.open(file_list[-1])
num_message = grb_in.messages
print('There are',num_message,'messages in this grib2 file')

There are 696 messages in this grib2 file


In [6]:
for message in grb_in:
    data = message.data()[0]
    meta = message.keys()
    if len(data.shape) != 2:
        print(message)

In [64]:
for file in file_list:
    grb = pg.open(file)
    with open(f'param_{file[-20:-16]}.csv','w') as f:
        for message in grb:
            output = f"{message['parameterNumber']}, {message['name']}, {message['units']}, {message['level']}, {message['paramId']}\n"
            f.write(output)
        f.close()

In [28]:
msg = grb_in.message(1)
print(msg['parameterNumber'],msg['parameterName'],msg['units'],msg['level'],msg['paramId'])

1 Pressure reduced to MSL Pa 0 260074


In [27]:
data[0]

array([[ -24.87,  -24.87,  -24.87, ...,  -24.87,  -24.87,  -24.87],
       [ -28.41,  -28.51,  -28.61, ...,  -28.23,  -28.29,  -28.35],
       [ -31.41,  -31.59,  -31.81, ...,  -31.01,  -31.15,  -31.27],
       ...,
       [-174.61, -175.03, -175.47, ..., -174.15, -174.39, -174.49],
       [-181.75, -181.97, -182.21, ..., -181.49, -181.61, -181.67],
       [-187.85, -187.85, -187.85, ..., -187.85, -187.85, -187.85]])

In [31]:
data[1]

array([[ 90.,  90.,  90., ...,  90.,  90.,  90.],
       [ 89.,  89.,  89., ...,  89.,  89.,  89.],
       [ 88.,  88.,  88., ...,  88.,  88.,  88.],
       ...,
       [-88., -88., -88., ..., -88., -88., -88.],
       [-89., -89., -89., ..., -89., -89., -89.],
       [-90., -90., -90., ..., -90., -90., -90.]])

In [32]:
data[2]

array([[  0.,   1.,   2., ..., 357., 358., 359.],
       [  0.,   1.,   2., ..., 357., 358., 359.],
       [  0.,   1.,   2., ..., 357., 358., 359.],
       ...,
       [  0.,   1.,   2., ..., 357., 358., 359.],
       [  0.,   1.,   2., ..., 357., 358., 359.],
       [  0.,   1.,   2., ..., 357., 358., 359.]])

## 读取多个文件的变量

In [24]:
for file in file_list:
    grb = pg.open(file)
    print(file[-20:-9],'  ',grb.messages)
    grb.close()

20140901_00    288
20150901_00    322
20160901_00    352
20170901_00    353
20180901_00    353
20190901_00    520
20200901_00    522
20210901_00    696
20220901_00    696


In [26]:
grb_2014 = pg.open(file_list[0])
grb_2022 = pg.open(file_list[-1])

In [36]:
ids_2014 = [message['parameterName'] + '_' + str(message['level']) for message in grb_2014]
ids_2022 = [message['parameterName'] + '_' + str(message['level']) for message in grb_2022]

In [37]:
ids_2014

['Geopotential height_10',
 'Temperature_10',
 'u-component of wind_10',
 'v-component of wind_10',
 'Absolute vorticity_10',
 '192_10',
 'Geopotential height_20',
 'Temperature_20',
 'u-component of wind_20',
 'v-component of wind_20',
 'Absolute vorticity_20',
 '192_20',
 'Geopotential height_30',
 'Temperature_30',
 'u-component of wind_30',
 'v-component of wind_30',
 'Absolute vorticity_30',
 '192_30',
 'Geopotential height_50',
 'Temperature_50',
 'u-component of wind_50',
 'v-component of wind_50',
 'Absolute vorticity_50',
 '192_50',
 'Geopotential height_70',
 'Temperature_70',
 'u-component of wind_70',
 'v-component of wind_70',
 'Absolute vorticity_70',
 '192_70',
 'Geopotential height_100',
 'Temperature_100',
 'Relative humidity_100',
 'Vertical velocity (pressure)_100',
 'u-component of wind_100',
 'v-component of wind_100',
 'Absolute vorticity_100',
 'Cloud mixing ratio_100',
 '192_100',
 'Geopotential height_150',
 'Temperature_150',
 'Relative humidity_150',
 'Vertic

In [38]:
for i, id_2022 in enumerate(ids_2022):
    try:
        j = ids_2014.index(id_2022)
        # 如果找到相同ID的消息，则输出其位置和ID
        print(f"Message {i} (ID: {id_2022}) in 2022 data corresponds to message {j} in 2014 data.")
    except ValueError:
        # 如果未找到相同ID的消息，则输出警告
        print(f"Warning: Message {i} (ID: {id_2022}) in 2022 data has no corresponding message in 2014 data.")
        continue

Message 0 (ID: Pressure reduced to MSL_0) in 2022 data corresponds to message 282 in 2014 data.
Message 8 (ID: 196_0) in 2022 data corresponds to message 257 in 2014 data.
Message 10 (ID: u-component of wind_0) in 2022 data corresponds to message 228 in 2014 data.
Message 11 (ID: v-component of wind_0) in 2022 data corresponds to message 229 in 2014 data.
Message 15 (ID: Temperature_1) in 2022 data corresponds to message 208 in 2014 data.
Message 16 (ID: Relative humidity_1) in 2022 data corresponds to message 260 in 2014 data.
Message 18 (ID: Vertical velocity (pressure)_1) in 2022 data corresponds to message 267 in 2014 data.
Message 20 (ID: u-component of wind_1) in 2022 data corresponds to message 265 in 2014 data.
Message 21 (ID: v-component of wind_1) in 2022 data corresponds to message 266 in 2014 data.
Message 23 (ID: 192_1) in 2022 data corresponds to message 209 in 2014 data.
Message 24 (ID: Geopotential height_2) in 2022 data corresponds to message 279 in 2014 data.
Message 

In [39]:
with open('output.txt', 'w') as f:
    # 在2022年的FNL数据文件中查找与2014年相同ID的消息
    for i, id_2022 in enumerate(ids_2022):
        try:
            j = ids_2014.index(id_2022)
            # 如果找到相同ID的消息，则输出其位置和ID，并将结果写入文本文件中
            output = f"Message {i} (ID: {id_2022}) in 2022 data corresponds to message {j} in 2014 data.\n"
            f.write(output)
        except ValueError:
            # 如果未找到相同ID的消息，则输出警告，并将结果写入文本文件中
            output = f"Warning: Message {i} (ID: {id_2022}) in 2022 data has no corresponding message in 2014 data.\n"
            f.write(output)